## Extreme precipitation in the Third Pole region : GPM, MSWEP and CNRR 

In [ ]:
import numpy as np
import os
import glob 
import random 

import matplotlib.pyplot as plt
import cartopy
from mpl_toolkits.basemap import Basemap
import matplotlib.colors as colors

import scipy
from scipy import ndimage
from scipy.stats import skew
from scipy.ndimage import label, generate_binary_structure
import matplotlib.pyplot as plt

from netCDF4 import Dataset
from collections import Counter 

import pandas as pd 

In [16]:
## import my own functions

import mcs_functions
from mcs_functions import read_in_netcdf

In [15]:
file = '/media/juli/Elements/GPM_IMERG_F_v05/GPM_finalrun/daily_files/GPM_IMERG_20170531_daily_acc_prec.nc4'
filename= 'GPM_IMERG_20170531_daily_acc_prec.nc4'
prec, lons, lats, date, time= read_in_netcdf(file, filename)


In [17]:
# extract only values above 3000 m a.s.l. 



In [ ]:
# calculate 99th percentile 
